In [122]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [123]:
#Function that uses inputs of url and headers to get a web page's html and return a BeautifulSoup of that html
def get_html(url, headers):
    response = requests.get(url, headers = headers, timeout=5)
    if response.status_code != 200:        
        print(page.status_code, url)
    else:  
        return BeautifulSoup(response.content, 'html.parser')

In [124]:
#Function that uses inputs of data in df convertible form, column names, and a csv filename to create a csv file
def create_csv(data, columns, filename):  
    df = pd.DataFrame(data, columns = columns)
    df.to_csv(filename)

In [183]:
#Creates a list urls with the offset values to access the list of 1000 top grossing movies of all time
urls = []
offset = 0
while offset <1000:
    urls.append("https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset={}".format(offset))
    offset+=200
urls

['https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=0',
 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=200',
 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=400',
 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=600',
 'https://www.boxofficemojo.com/chart/top_lifetime_gross/?offset=800']

In [184]:
#Code to scrape Box Office Mojo top 1000 grossing domestic films of all time and save to csv

#Defines function input variables, as well as creates empty lists that will contain the values for each df column
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
mov_titles = []
dom_gross = []
year_list = []
columns = ['Title','Year','Domestic Gross']
filename = 'Movies.csv'
#For loop that iterates through the urls that contain pages of the 1000 top grossing movies of all time list
for url in urls:
#Function call that returns the BeautifulSoup of the html for the given url
    soup = get_html(url, headers)
#Finds gross by movie in the BeautifulSoup, extracts it from the html and appends it to the gross column list
    gross_values = soup.find_all('td',class_='a-text-right mojo-field-type-money')
    for data in gross_values:
        dom_gross.append(str(data).split('>')[1][:-4])
#Finds release year by movie in the BeautifulSoup, extracts it from the html and appends it to the year column list
    table = soup.find(id='table')
    year_data = table.find_all('td',class_='a-text-left mojo-field-type-year')
    for data in year_data:
        if (str(data).split('>')[2][:-3]):
            year_list.append(str(data).split('>')[2][:-3])
        else:
            year_list.append(str(data).split('>')[1][:-4])
#Finds title by movie in the BeautifulSoup, extracts it from the html and appends it to the movie title list
    title_data = table.find_all('td',class_='a-text-left mojo-field-type-title')
    for data in title_data:
            mov_titles.append(str(data).split('>')[2][:-3])

#Creates a list of lists from the df column value lists with each index list containing values for a df row
data = []
for index in (range(0,len(mov_titles))):
    data.append([mov_titles[index],year_list[index],dom_gross[index]])
#Function call that creates a csv from the data in list of lists form
create_csv(data, columns, filename)

In [222]:
#Code to scrape The Numbers' list of movie genres

#Calls function to get webpage html
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
url = 'https://www.the-numbers.com/market/genres'
genre_list_request = requests.get(url, headers=headers, timeout=5)

#Creates BeautifulSoup from html and finds links containing movie genre names
genre_list_soup = BeautifulSoup(genre_list_request.content, 'html.parser')
genre_list_table = genre_list_soup.find('table')
genre_list_list = genre_list_table.find_all('a')

#Extracts genre names from link html and appends to a genre pre-list
genre_prelist = []
for item in genre_list_list:
    genre_prelist.append(str(item).split('>')[1][:-3])

#Corrects genre pre-list for characters that vary from the scraped genre list to the urls for those genres
genre_list = []
for item in genre_prelist:
    updated = item.replace('/','-or-')
    updated = updated.replace(' ','-')
    genre_list.append(updated)



    

In [223]:
#Code to scrape The Numbers annual box office data by genre and save to csv

#Defines function input variables, as well as creates empty lists that will contain the values for each df column
genres = []
year = []
movie_count = []
share = []
gross = []
tickets = []
inflated_gross = []
columns = ['Genre','Year', 'Movie Count', 'Share','Gross','Tickets', 'Inflated Gross']
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
filename = 'Genres.csv'
#For loop that iterates through the genres to be scraped
for genre in genre_list:
    print(genre)
#Generates genre name-based url to scrape and returns a BeautifulSoup of the html for that url
    url = 'https://www.the-numbers.com/market/genre/{}'.format(genre)
    soup = get_html(url, headers)
#Creates a raw_data list of html snippets with relevant data points
    tables = soup.find_all('table')
    raw_data = tables[1].find_all('td')
#Splits and slices raw data to extract year and annual movie count values per anum 
    sliced_year_count = []
    for item in raw_data:
        sliced_year_count.append(str(item).split('>')[2][:-3])
#Indexes, splits and slices raw data to extract market share values per anum and append to the share df column list
    unsliced_share = []
    iter_length = int(len(raw_data)/8)
    for index in range(0,iter_length):
        unsliced_share.append(raw_data[8*index+2])
    for item in unsliced_share:
        share.append(str(item).split('>')[3][0:-3])
#Splits and slices raw data to extract gross, tickets and inflated gross values per anum
    sliced_gross = []
    for item in raw_data:
        sliced_gross.append(str(item).split('>')[1][:-4])
#Creates an ordered list of all extracted data points
    data_points = []
    for index in (range(0,iter_length)):
        data_points.append(sliced_year_count[8*index])
        data_points.append(sliced_year_count[8*index+1])
        data_points.append(sliced_gross[8*index+3])
        data_points.append(sliced_gross[8*index+4])
        data_points.append(sliced_gross[8*index+5])
#Appends data points to the list corresponding to their df column
    for index in range(0,int(len(data_points)/5)):
        year.append(data_points[5*index])
        movie_count.append(data_points[5*index+1])
        gross.append(data_points[5*index+2])
        tickets.append(data_points[5*index+3])
        inflated_gross.append(data_points[5*index+4])
        genres.append(genre)
#Creates a list of lists from the df column value lists with each index list containing values for a df row
data = []
for index in range(0,len(year)):
    data.append([genres[index], year[index], movie_count[index], share[index], gross[index], tickets[index], inflated_gross[index]])
#Function call that creates a csv from the data in list of lists form
create_csv(data, columns, filename)

    

Adventure
Action
Drama
Comedy
Thriller-or-Suspense
Horror
Romantic-Comedy
Musical
Documentary
Black-Comedy
Western
Concert-or-Performance
Multiple-Genres
Reality


In [232]:
#Code to scrape The Numbers' list of movie studios

#Calls function to get webpage html
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
url = 'https://www.the-numbers.com/market/distributors'
dist_list_request = requests.get(url, headers=headers, timeout=5)

#Creates BeautifulSoup from html and finds links containing movie studio names
dist_list_soup = BeautifulSoup(dist_list_request.content, 'html.parser')
dist_list_table = dist_list_soup.find('table')
dist_list_list = dist_list_table.find_all('a')

#Extracts movie studio names from link html and appends to a genre pre-list
dist_prelist = []
for item in dist_list_list:
    dist_prelist.append(str(item).split('>')[1][:-3])

#Corrects genre pre-list for variations between scraped genre list and the urls for those genres
dist_list = []
for item in dist_prelist:
    updated = item.replace('/','-')
    updated = updated.replace(' - ', '-')
    updated = updated.replace(' ','-')
    updated = updated.replace('.','')
    updated = updated.replace('Alliance-Films','Alliance-Atlantis')
    updated = updated.replace('The-Orchard','Orchard-The')
    dist_list.append(updated)

#Slices distributor list to top 100 distributors
dist_list = dist_list[0:100]

In [233]:
#Code to scrape The Numbers annual box office data by distributor and save to csv

#Defines function input variables, as well as creates empty lists that will contain the values for each df column
dists = []
year = []
movie_count = []
share = []
gross = []
tickets = []
inflated_gross = []
columns = ['Distributor', 'Year', 'Movie Count', 'Share','Gross','Tickets', 'Inflated Gross']
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
filename = 'Distributors.csv'
#For loop that iterates through the distributors to be scraped
for dist in dist_list:
    print(dist)
#Generates distributor name-based url to scrape and returns a BeautifulSoup of the html for that url
    url = 'https://www.the-numbers.com/market/distributor/{}'.format(dist)
    soup = get_html(url, headers)
#Creates a raw_data list of html snippets with relevant data points
    tables = soup.find_all('table')
    raw_data = tables[1].find_all('td')
#Splits and slices raw data to extract year and annual movie count values per anum 
    sliced_year_count = []
    for item in raw_data:
        sliced_year_count.append(str(item).split('>')[2][:-3])
#Indexes, splits and slices raw data to extract market share values per anum and append to the share df column list
    unsliced_share = []
    iter_length = int(len(raw_data)/8)
    for index in range(0,iter_length):
        unsliced_share.append(raw_data[8*index+2])
    for item in unsliced_share:
        share.append(str(item).split('>')[3][0:-3])
#Splits and slices raw data to extract gross, tickets and inflated gross values per anum
    sliced_gross = []
    for item in raw_data:
        sliced_gross.append(str(item).split('>')[1][:-4])
    iter_length = int(len(sliced_year_count)/8)
#Creates an ordered list of all extracted data points
    data = []
    for index in (range(0,iter_length)):
        data.append(sliced_year_count[8*index])
        data.append(sliced_year_count[8*index+1])
        data.append(sliced_gross[8*index+3])
        data.append(sliced_gross[8*index+4])
        data.append(sliced_gross[8*index+5])
#Appends data points to the list corresponding to their df column
    for index in range(0,int(len(data)/5)):
        year.append(data[5*index])
        movie_count.append(data[5*index+1])
        gross.append(data[5*index+2])
        tickets.append(data[5*index+3])
        inflated_gross.append(data[5*index+4])
        dists.append(dist)
#Creates a list of lists from the df column value lists with each index list containing values for a df row        
data= []
for index in range(0,len(year)):
    data.append([dists[index], year[index], movie_count[index], share[index], gross[index], tickets[index], inflated_gross[index]])
#Function call that creates a csv from the data in list of lists form
create_csv(data, columns, filename)


Walt-Disney
Warner-Bros
Sony-Pictures
Universal
20th-Century-Fox
Paramount-Pictures
Lionsgate
New-Line
Dreamworks-SKG
Miramax
MGM
Fox-Searchlight
Focus-Features
Weinstein-Co
Summit-Entertainment
Sony-Pictures-Classics
STX-Entertainment
Miramax-Dimension
Relativity
Open-Road
A24
Newmarket-Films
Roadside-Attractions
IFC-Films
CBS-Films
FilmDistrict
Artisan
Paramount-Vantage
Overture-Films
United-Artists
IMAX-Films
USA-Films
Gramercy
Weinstein-Dimension
Pure-Flix-Entertainment
Bleecker-Street
Magnolia-Pictures
Picturehouse
Polygram
Warner-Independent
MacGillivray-Freeman-Films
Focus-Rogue-Pictures
Neon
Fine-Line
Entertainment-Studios-Motion-Pictures
Samuel-Goldwyn-Films
Eros-Entertainment
Sony-TriStar
Freestyle-Releasing
October-Films
Annapurna-Pictures
Broad-Green-Pictures
UTV-Communications
Aviron-Pictures
Yash-Raj-Films
Orion-Pictures
Music-Box-Films
FUNimation
Savoy
Rocky-Mountain-Pictures
Filmways-Pictures
20th-Century-Studios
ThinkFilm
National-Geographic-Entertainment
Yari-Film-Gro

In [195]:
#Code to scrape Box Office Mojo top 1000 grossing domestic films of all time and save to csv

#Defines function input variables, as well as creates empty lists that will contain the values for each df colum
url = 'https://www.boxofficemojo.com/franchise/?ref_=bo_nb_gs_secondarytab'
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
franch_titles = []
franch_gross = []
franch_releases = []
top_releases = []
releases_gross = []
columns = ['Franchise','Total Gross','Releases','Top Release','Top Release Gross']
filename = "Franchises.csv"

#Function call that returns the BeautifulSoup of the html for the given url
soup = get_html(url, headers)
#Finds gross amount by franchise in the BeautifulSoup and extracts the values from the html
franch_gross_values = soup.find_all('td',class_='a-text-right mojo-field-type-money mojo-sort-column mojo-estimatable')
for data in franch_gross_values:
    franch_gross.append(str(data).split('>')[1][:-4])  
#Finds gross amount by top release in the BeautifulSoup and extracts the values from the html
releases_gross_values = soup.find_all('td',class_='a-text-right mojo-field-type-money mojo-estimatable')
for data in releases_gross_values:
    releases_gross.append(str(data).split('>')[1][:-4])
#Finds number of releases by franchise in the BeautifulSoup and extracts the values from the html
franch_releases_values = soup.find_all('td',class_='a-text-right mojo-field-type-positive_integer')
for data in franch_releases_values:
    franch_releases.append(str(data).split('>')[1][:-4])
#Finds the franchise names and top releases by franchise in the BeautifulSoup and extracts the values from the html
table = soup.find(id='table')
other_data = table.find_all('a',class_='a-link-normal')
other= []
for data in other_data:
    other.append(str(data).split('>')[1][:-3])
#Seperates the list of franchise names and top releases by franchise into seperate lists for the two sets of values
other = other[3:]
it_len = int(len(other)/2)
for index in (range(0,it_len)):
    franch_titles.append(other[2*index])
    top_releases.append(other[2*index+1])
#Creates a list of lists from the column value lists with each index list containing values for a df row
data = []
for index in (range(0,it_len)):
    data.append([franch_titles[index],franch_gross[index],franch_releases[index],top_releases[index],releases_gross[index]])
#Function call that creates a csv from the data in list of lists form
create_csv(data, columns, filename)

SystemExit: 

/Users/blantj/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
